import pandas as pd

In [3]:
import pandas as pd

In [4]:
domus = pd.read_csv(r"C:\Users\Dell\Desktop\Restaurant\Domus Lounge Bar.csv", error_bad_lines=False);
hadjigeorgakis = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Ethnological Museum House of Hadjigeorgakis Kornesios.csv', error_bad_lines=False);
folk_mus = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Cyprus Folk Art Museum.csv', error_bad_lines=False);
nat_struggle = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\The National Struggle Museum.csv', error_bad_lines=False);
liberty = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Liberty Monument.csv', error_bad_lines=False);
archbishop_fd = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Archbishop Makarios III Cultural Foundation.csv', error_bad_lines=False);

tripadv_review = pd.concat([domus, hadjigeorgakis, folk_mus, nat_struggle, liberty, archbishop_fd])
tripadv_review

,r_points,r_text,place
0,50,"If you want romance, pleasant atmosphere ... G...",Domus Lounge Bar
1,40,DomusArrived in the dark and being a first tim...,Domus Lounge Bar
2,50,Great place for friends and couples. Delicious...,Domus Lounge Bar
3,50,Superb restaurant in the old town with very go...,Domus Lounge Bar
4,50,"Positioned in the Old Town of Nicosia, this re...",Domus Lounge Bar
...,...,...,...
17,50,One of the very few places in the world to vie...,Archbishop Makarios III Cultural Foundation
18,40,If you aren't interested in Byzantine art and ...,Archbishop Makarios III Cultural Foundation
19,50,This museum displays more than 300 stunningly ...,Archbishop Makarios III Cultural Foundation
20,50,"If you like Christian Art, you'll love this mu...",Archbishop Makarios III Cultural Foundation


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [8]:
processed_reviews = tripadv_review['r_text'].astype(str).map(preprocess)
processed_reviews

0     [want, romance, pleasant, atmosphere, good, vi...
1     [domusarrived, dark, time, visitor, initially,...
2     [great, place, friends, couples, delicious, me...
3     [superb, restaurant, town, good, food, highly,...
4     [positioned, town, nicosia, restaurant, offers...
                            ...                        
17    [places, world, view, greek, orthodox, ancient...
18    [aren, interested, byzantine, icons, bother, b...
19    [museum, displays, stunningly, preserved, icon...
20    [like, christian, love, museum, gallery, greek...
21                      [place, visited, pure, history]
Name: r_text, Length: 364, dtype: object

In [9]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]]


In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=400,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [19]:
from pprint import pprint

In [20]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"place" + 0.010*"monument" + 0.010*"domus" + 0.009*"time" + '
  '0.008*"nice" + 0.008*"good" + 0.007*"cyprus" + 0.006*"worth" + '
  '0.006*"restaurant" + 0.005*"city"'),
 (1,
  '0.021*"food" + 0.016*"good" + 0.014*"great" + 0.012*"restaurant" + '
  '0.012*"service" + 0.011*"nice" + 0.009*"excellent" + 0.009*"visit" + '
  '0.008*"atmosphere" + 0.008*"museum"'),
 (2,
  '0.018*"museum" + 0.013*"history" + 0.011*"cyprus" + 0.008*"visit" + '
  '0.008*"food" + 0.008*"monument" + 0.007*"struggle" + 0.007*"cypriot" + '
  '0.006*"time" + 0.006*"good"'),
 (3,
  '0.015*"food" + 0.011*"place" + 0.010*"atmosphere" + 0.009*"excellent" + '
  '0.009*"service" + 0.008*"restaurant" + 0.008*"museum" + 0.007*"nicosia" + '
  '0.007*"great" + 0.007*"dinner"'),
 (4,
  '0.012*"great" + 0.011*"nicosia" + 0.009*"place" + 0.009*"museum" + '
  '0.008*"british" + 0.006*"food" + 0.006*"monument" + 0.006*"greek" + '
  '0.006*"people" + 0.006*"worth"')]


In [21]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.073964  0.022645       1        1  35.619442
3     -0.074539 -0.021323       2        1  20.799181
2      0.063571  0.056100       3        1  17.277697
4      0.051089 -0.089148       4        1  14.187852
0      0.033842  0.031725       5        1  12.115830, topic_info=          Term       Freq       Total Category  logprob  loglift
1859  monument  36.000000   36.000000  Default  30.0000  30.0000
1030   history  30.000000   30.000000  Default  29.0000  29.0000
12       great  77.000000   77.000000  Default  28.0000  28.0000
182      place  71.000000   71.000000  Default  27.0000  27.0000
1484    museum  84.000000   84.000000  Default  26.0000  26.0000
...        ...        ...         ...      ...      ...      ...
85        food   4.732040  121.460037   Topic5  -5.4405  -1.1346
1484    museum   4.323066   84.337463   Topic5  -5.5309  -0.8602
623       look   2.927295   11.856097   Topic5  -5.9208   0.7119
8       dishes   3.069067   20.209938   Topic5  -5.8735   0.2259
1662  struggle   2.937757   29.304840   Topic5  -5.9172  -0.1894

[384 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         4  0.358228  according
0         5  0.358228  according
2449      5  0.723371     admire
717       5  0.478592      ahead
194       1  0.388299    amazing
...     ...       ...        ...
281       3  0.212317      worth
281       4  0.212317      worth
281       5  0.185777      worth
399       1  0.892285     worths
2249      5  0.477050       ομως

[668 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 5, 1])